### Table of Contents

01. Importing Libraries

02. Importing Data Sets

03. Defining Functions

04. Testing price_label Function on Practice Data Frame

05. Test loc Method to Create a Flagging Column

06. loc Method to Create a Flagging Column for Price

07. Trouble Shooting application of loc Method for Whole Data Frame

08. Updating busiest_day Column Based on Client Feedback

09. Creating busiest_period_of_day column

# 01. Importing Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02. Importing Data Sets

In [2]:
# Creating a path variable as a short cut for future importing of data
path = r'/Users/tuor/DA Projects/230321 Instacart Basket Analysis'

In [3]:
# Importing orders_products_merged data frame
df_ord_prod_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [5]:
# Subset environment of orders_products_merged to test execution of edits
df = df_ord_prod_merge[:1000000]

# 03. Defining Functions

In [6]:
# Price Label function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

# 04. Testing price_label Function on Practice Data Frame

In [7]:
# Applying price_label function to create a column to bucket prices
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/lr/bnypmrgd10lfz8x9hx1x8w9w0000gn/T/ipykernel_57351/2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [8]:
# counting the values in the price_range column
df['price_range'].value_counts(dropna = False)

Mid-range product    659373
Low-range product    331283
High range             9344
Name: price_range, dtype: int64

In [9]:
# Checking the max value in prices
df['prices'].max()

24.5

# 05. Test loc Method to Create a Flagging Column

In [13]:
# If prices > 15 using loc method to label flagging column
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [11]:
# If prices > 5 and <= 15 using loc method to label flagging column
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [12]:
# If prices <= 5 using the loc method to label flagging column
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
# Counting new flagging column to confirm results
df['price_range_loc'].value_counts(dropna = False)

Mid-range product     659373
Low-range product     331283
High-range product      9344
Name: price_range_loc, dtype: int64

# 06. loc Method to Create a Flagging Column for Price

In [18]:
# If prices > 15 using loc method to label flagging column
df_ord_prod_merge.loc[df_ord_prod_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [28]:
# If prices > 5 and <= 15 using loc method to label flagging column
df_ord_prod_merge.loc[(df_ord_prod_merge['prices'] <= 15) & (df_ord_prod_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [20]:
# If prices <= 5 using the loc method to label flagging column
df_ord_prod_merge.loc[df_ord_prod_merge['prices'] <= 5, 'price_range_loc'] = 

In [29]:
# Counting new flagging column to confirm results
df_ord_prod_merge['price_range_loc'].value_counts(dropna = False)

Mid-range product     21890146
Low-range product     10126384
High-range product      417682
Name: price_range_loc, dtype: int64

# 07. Trouble Shooting application of loc Method for Whole Data Frame

In [25]:
# create a boolean mask for NaN values in price_range_loc column
mask = df_ord_prod_merge['price_range_loc'].isna()

In [26]:
# filter the data frame using boolean indexing
df_filtered = df_ord_prod_merge.loc[mask]

In [27]:
df_filtered.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,merge_indicator,price_range_loc
1000000,2029,Swirl Thick Slice Brown Sugar Cinnamon Bread,112,3,8.3,2442034,66203,11,6,15,6.0,8,0,both,both,NaN
1000001,2029,Swirl Thick Slice Brown Sugar Cinnamon Bread,112,3,8.3,1917510,67065,1,5,13,NaN,3,0,both,both,NaN
1000002,2029,Swirl Thick Slice Brown Sugar Cinnamon Bread,112,3,8.3,230170,67065,2,1,17,10.0,4,1,both,both,NaN
1000003,2029,Swirl Thick Slice Brown Sugar Cinnamon Bread,112,3,8.3,1083075,67065,3,5,11,30.0,7,1,both,both,NaN
1000004,2029,Swirl Thick Slice Brown Sugar Cinnamon Bread,112,3,8.3,1482805,67065,4,3,16,26.0,7,1,both,both,NaN


Found an error in my loc code for flagging Mid-range product where it was calling from the wrong data frame resulting in a significan't number of NaN results.

# 08. Creating busiest_day Column

In [34]:
df_ord_prod_merge['order_day_of_week'].value_counts(dropna = False)

0    6209632
1    5665830
6    4500246
2    4217766
5    4209449
3    3844096
4    3787193
Name: order_day_of_week, dtype: int64

In [38]:
# For Loop to create a flag column to note level of business
result = []

for value in df_ord_prod_merge["order_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [44]:
result

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Bus

In [46]:
# Creating busiest_day column and filling it with result
df_ord_prod_merge['busiest_day'] = result

In [47]:
df_ord_prod_merge['busiest_day'].value_counts(dropna = False)

Regularly busy    22437387
Busiest day        6209632
Least busy         3787193
Name: busiest_day, dtype: int64

# 08. Updating busiest_day Column Based on Client Feedback

In [58]:
# For Loop to update a flag column to note level of business
result = []

for value in df_ord_prod_merge["order_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 3:
    result.append("2nd Least busy") 
  elif value == 4:
    result.append("Least busy")   
  else:
    result.append("Regularly busy")

In [60]:
result

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 '2nd Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 '2nd Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 '2nd Least busy',
 'Least busy',
 'Regularly busy',
 '2nd Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 '2nd Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 '2nd Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Bus

In [61]:
# Updating busiest_day column and filling it with result
df_ord_prod_merge['busiest_day'] = result

In [62]:
# Counting busiest_day column to check work
df_ord_prod_merge['busiest_day'].value_counts(dropna = False)

Regularly busy    18593291
Busiest day        6209632
2nd Least busy     3844096
Least busy         3787193
Name: busiest_day, dtype: int64

In [63]:
# Renaming column busiest_day to busiest_days inplace
df_ord_prod_merge.rename(columns={'busiest_day': 'busiest_days'}, inplace=True)

In [64]:
df_ord_prod_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,merge_indicator,price_range_loc,busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,both,Mid-range product,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,both,Mid-range product,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,both,Mid-range product,Busiest day
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,both,Mid-range product,2nd Least busy
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,both,Mid-range product,Least busy


# 09. Creating busiest_period_of_day column

In [65]:
# Assessing the busiest hours
df_ord_prod_merge['order_hour_of_day'].value_counts(dropna = False)

10    2764390
11    2738585
14    2691563
15    2664522
13    2663272
12    2620800
16    2537469
9     2456661
17    2089452
8     1719952
18    1637922
19    1259382
20     977017
7      891928
21     796362
22     634737
23     402612
6      290770
0      218942
1      115786
5       88057
2       69431
4       53283
3       51317
Name: order_hour_of_day, dtype: int64

In [73]:
# For Loop to create a flag column to note level of business by hour
result = []

for value in df_ord_prod_merge["order_hour_of_day"]:
  if value == 3:
    result.append("Fewst orders")
  elif value == 10:
    result.append("Most orders")   
  else:
    result.append("Average orders")

In [74]:
result

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Fewst orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders

In [75]:
# Creating busiest_period_of_day column and filling it with result
df_ord_prod_merge['busiest_period_of_day'] = result

In [76]:
# Counting busiest_day column to check work
df_ord_prod_merge['busiest_period_of_day'].value_counts(dropna = False)

Average orders    29618505
Most orders        2764390
Fewst orders         51317
Name: busiest_period_of_day, dtype: int64

In [77]:
# exporting to pickle
df_ord_prod_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', '230328_order_products_merged.pkl'))